# Segmenting and Clustering Neighborhoods in Toronto

## 1. Read Postal Codes and Neighborhood Data
In this section,we read the datas of Toronto postal codes from mentioned wiki page and create a dataframe. We exclude the entries which has 'Borough' as 'Not Assigned'. We also populate the missing or not assigned 'Neighborhood' values with the corresponding 'Borough' values. The we combine all entries for a postal code with Negihborhood column having comma separated values.    
I have used pandas for reading the html page. Upoon examining the source pf the page, we get the class atrribute of the html table which can be used to filter out unwanted data.

In [9]:
!conda install -c conda-forge folium --yes
import pandas as pd
import folium
import requests
folium.__version__
#!conda install -c conda-forge geocoder
html_df_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',attrs={'class' : "wikitable sortable"})
print('read table from html page')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.10.0              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTAL

In [10]:
df = html_df_list[0]
df.columns=['Postal Code','Borough','Neighborhood']
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'],axis=1)
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df.shape)
df.head(10)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## 2. Read geo co-ordinates for postal codes
For this task, I tried to use the geocode library as suggested, but it returned None for most of the times. So I used the csv shared in the assignment for the data.

In [12]:
#import geocoder
#lat_lng_coords = None
#while(lat_lng_coords is None):
# g = geocoder.google('{}, Toronto, Ontario'.format('M8Z'))
# lat_lng_coords = g.latlng

In [13]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 3. Prepare required dataset
We need to combine both geo and neighborhood data. To reduce the number of entries for FourSquare API, I havs filtered only entries with Borough names having 'Toronto'. This is as suggested in the assignment description.

In [14]:
df_joined = df.join(geo_df.set_index('Postal Code'),on = 'Postal Code')

In [15]:
df_toronto = df_joined[df_joined['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


## 4. Explore all the postal codes with FourSquare API

In [16]:
#Copied from the ungraded assignment.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
 # @hidden_cell
CLIENT_ID = 'REBLXQGQCD1KBPP3ZQUNQEGQKQ3HZ4K2TKC1NZMKKQULI1HL' # your Foursquare ID
CLIENT_SECRET = '1FGC5JK3XMZ4NOXWCY44LTHIED3D3FSSVBIGZP42CIXJA43N' # your Foursquare Secret

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: REBLXQGQCD1KBPP3ZQUNQEGQKQ3HZ4K2TKC1NZMKKQULI1HL
CLIENT_SECRET:1FGC5JK3XMZ4NOXWCY44LTHIED3D3FSSVBIGZP42CIXJA43N


In [18]:
VERSION = '20180605' # Foursquare API version
radius = 1000
LIMIT = 30


In [19]:
toronto_venues = getNearbyVenues(names=df_toronto['Postal Code'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
toronto_venues

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,M4K,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
6,M4K,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,M4K,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,M4K,43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
9,M4K,43.679557,-79.352188,Louis Cifer Brew Works,43.677663,-79.351313,Brewery


## 5. Get top 5 venues within a postal code by count

In [40]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6016b52f95a646e4a3301b0467b7a24b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='nO9TaAqp7gNr8GjZAQktthFds0g8sP8rBCfigrIHQGxf',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6016b52f95a646e4a3301b0467b7a24b.get_object(Bucket='courseracapstone-donotdelete-pr-c10sqgpkgkbykm',Key='value_cat.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

value_cat = pd.read_csv(body,encoding = "ISO-8859-1")
value_cat.head()


,Value Category,Classification
0,Airport,Airport
1,Airport Food Court,Airport
2,Airport Gate,Airport
3,Airport Lounge,Airport
4,Airport Service,Airport


In [93]:
import numpy as np
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#print(toronto_venues['Venue Category'].unique())
toronto_venues1 = toronto_venues.join(value_cat.set_index('Value Category'),on='Venue Category')
toronto_venues1
t1 = toronto_venues1.groupby(['Postal Code','Classification']).size()
t2 = pd.DataFrame(t1).reset_index()
t2.columns = ['Postal Code','Venue Classification','Count']
t4 = pd.DataFrame(toronto_venues1.groupby('Classification').size()).reset_index()
t3 = pd.pivot_table(t2, values='Count', index=['Postal Code'],columns=['Venue Classification'], aggfunc=np.sum, fill_value=0)
x=pd.DataFrame(t3)
x

There are 188 uniques categories.


Venue Classification,Academic,Airport,Business,Commercial,Eatery,Entertainment,Exhibitions,Fitness,Grocery,Hangout,Leisure Shop,Recreational,Residence,Restaurant - American,Restaurant - Asian,Restaurant - Europian,Restaurant - General,Sport,Transit
Postal Code,,,,,,,,,,,,,,,,,,,
M4E,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0
M4K,0,0,0,3,4,0,0,2,5,2,0,1,0,0,1,10,2,0,0
M4L,0,0,0,1,8,1,0,1,2,2,0,1,0,1,1,1,1,0,0
M4M,0,0,1,2,8,0,0,1,5,2,0,1,1,2,2,2,3,0,0
M4N,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
M4P,0,0,1,1,3,0,0,1,0,0,0,1,0,0,0,0,0,0,0
M4R,0,0,1,4,4,0,0,2,2,0,2,1,0,1,1,0,2,0,0
M4S,0,0,0,1,8,1,0,2,5,1,1,1,0,0,4,3,3,0,0
M4T,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


## 6. Perform segmentation with K-Means clustering

In [94]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn import preprocessing
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
x.reset_index(inplace=True)
x_data = x.drop(columns=['Postal Code'])
x_scaled = preprocessing.StandardScaler().fit_transform(x_data)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(x_scaled)

# add clustering labels
x.insert(0, 'Cluster Labels', kmeans.labels_)
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([4, 0, 0, 0, 4, 4, 0, 0, 4, 0], dtype=int32)

In [95]:

#rslt.drop(columns=['top3','top4','top5','Postal Code_num','top1_num','top2_num','top3_num','top4_num','top5_num'],inplace=True)
#rslt.insert(0, 'Cluster Labels', kmeans.labels_)

# manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = x.join(df_toronto.set_index('Postal Code'), on='Postal Code')
#manhattan_merged.reset_index(inplace=True)
manhattan_merged.head() # check the last columns!

,Cluster Labels,Postal Code,Academic,Airport,Business,Commercial,Eatery,Entertainment,Exhibitions,Fitness,...,Restaurant - American,Restaurant - Asian,Restaurant - Europian,Restaurant - General,Sport,Transit,Borough,Neighborhood,Latitude,Longitude
0,4,M4E,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,East Toronto,The Beaches,43.676357,-79.293031
1,0,M4K,0,0,0,3,4,0,0,2,...,0,1,10,2,0,0,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,0,M4L,0,0,0,1,8,1,0,1,...,1,1,1,1,0,0,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,0,M4M,0,0,1,2,8,0,0,1,...,2,2,2,3,0,0,East Toronto,Studio District,43.659526,-79.340923
4,4,M4N,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,Central Toronto,Lawrence Park,43.728020,-79.388790


In [96]:
# create map
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Postal Code'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
def get_weight(cls,count):
    #print(cls)
    row = t4[t4['Classification']==cls]
    wt = count/row.iloc[0]['Count']
    #print(wt)
    return wt
t4.columns = ['Classification','Count']
print(t4)
pd.set_option('display.width', 250)
for i in range(0,kclusters):
    df = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == i, :]
    df = df.drop(columns=['Latitude','Longitude','Cluster Labels'])
    df = df.loc[:, (df != 0).any(axis=0)]
    df_sum = df.sum(numeric_only=True) 
    df_pd = pd.DataFrame(df_sum)
    df_pd.reset_index(inplace=True)
    df_pd.columns=['Classification','Count']
    df_pd['Class Wt'] = df_pd.apply(lambda x:get_weight(x['Classification'],x['Count']),axis=1)
    df_pd.sort_values(by=['Class Wt'],ascending=False,inplace=True)
    print('-------Start cluster: '+str(i) +'---------')
    print(df_pd)
    print('-------End cluster: '+str(i)+ '\ttotal postal codes: ' + str(len(df.index))+'---------')

           Classification  Count
0                Academic      2
1                 Airport      8
2                Business     23
3              Commercial     52
4                  Eatery    213
5           Entertainment     28
6             Exhibitions     19
7                 Fitness     33
8                 Grocery     91
9                 Hangout     83
10           Leisure Shop     10
11           Recreational     35
12              Residence      5
13  Restaurant - American     37
14     Restaurant - Asian     57
15  Restaurant - Europian     49
16   Restaurant - General     67
17                  Sport      8
18                Transit      7
-------Start cluster: 0---------
           Classification  Count  Class Wt
8            Leisure Shop      8  0.800000
13  Restaurant - Europian     38  0.775510
1              Commercial     38  0.730769
12     Restaurant - Asian     39  0.684211
6                 Grocery     61  0.670330
3           Entertainment     18  0.642857
2     

## 7.Deduction from the clustered data.    
  ### Cluster 0: Light Commercial, light recreational    
   The first cluster has few bakeries, cafes and restaurants, which along with an auto-shop and rail station suggests places of small number of commercial or retail shops. A park makes it having a light recreational flavor.    

   ### Cluster 1: High recreational, light commercial      
   Few parks, playground suggest this cluster to be a high recreational area. Jwelary shop and Greer restaurants makes it light commercial. M4K seems dominated by Greek population with high number of Greek restaurants.    


###    Cluster 2: Shopping, hangout      
This cluster is of high commercial points, with many shopping and hangout places as clothing stores, gift shops, bars, breweries, cafes and so on.     

###   Cluster 3: High retail business, light recreational      
This cluster includes many retail businesses such as grocery, home service,  bakery, health food stores. The business side also includes few cafes and restaurants. It includes few recreational entries as park and garden.    

###    Cluster 4: Airport and Eateries     
This cluster include Airport and related venues as top entry. Other two postal codes include few Eateries.     

  